In [11]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import numpy as np
import pickle
from PIL import Image
import random
import seaborn as sns

In [37]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K

from keras.models import Sequential, Model

from keras.layers import Input, Layer, Dense, Conv2D, Conv2DTranspose
from keras.layers import Flatten, Reshape, MaxPooling2D, UpSampling2D
from keras.layers import Dropout, BatchNormalization
from keras.layers import Activation, LeakyReLU

from keras import regularizers, initializers
from keras.regularizers import l2
from keras.initializers import RandomUniform, Constant
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping

from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## EEG

In [4]:
from sklearn.model_selection import train_test_split

def load_eeg(path):
    eeg = pickle.load(open(f'{path}/data.pkl', 'rb'), encoding='latin1')
    x_train, x_test = eeg['x_train'], eeg['x_test']
    y_train, y_test = eeg['y_train'], eeg['y_test']
    x_train, x_vali, y_train, y_vali = train_test_split(x_train, y_train, test_size=0.20, random_state=42)
    
    eeg = {'x_train': x_train, 'y_train': y_train,
                  'x_vali': x_vali, 'y_vali': y_vali,
                  'x_test': x_test, 'y_test': y_test}

    return eeg

In [6]:
obj_eeg = load_eeg('datasets/thoughtviz/data/eeg/image')

In [7]:
obj_eeg['x_train'].shape, obj_eeg['y_train'].shape

((36312, 14, 32, 1), (36312, 10))

In [8]:
class ThoughtViz:
  def __init__(self):
    channels, observations, num_classes = 14, 32, 10
    model = Sequential()
    model.add(BatchNormalization(input_shape=(channels, observations, 1)))
    model.add(Conv2D(32, (1, 4), activation='relu'))
    model.add(Conv2D(25, (channels, 1), activation='relu'))
    model.add(MaxPooling2D((1, 3)))
    model.add(Conv2D(50, (4, 25), activation='relu', data_format='channels_first'))
    model.add(MaxPooling2D((1, 3)))
    model.add(Conv2D(100, (50, 2), activation='relu'))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))
    model._name = 'classifier_encoder_thoughtviz'
    self.model = model

In [9]:
eeg_classifier = ThoughtViz().model
eeg_classifier.summary()

Model: "classifier_encoder_thoughtviz"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 14, 32, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 14, 29, 32)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 29, 25)         11225     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 9, 25)         0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 6, 1)          5050      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 

In [10]:
eeg_classifier.compile(optimizer=keras.optimizers.legacy.SGD(learning_rate=1 * (10 ** -4), momentum=0.9, decay=1 * (10 ** -6)), 
                       loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.005, patience=50, restore_best_weights=True, verbose=1)
callbacks = [early_stopping_callback]

eeg_classifier.fit(obj_eeg['x_train'], obj_eeg['y_train'], epochs=30, validation_data=(obj_eeg['x_vali'], obj_eeg['y_vali']), batch_size=32, callbacks=callbacks)

Epoch 1/30
1135/1135 [==============================] - 31s 13ms/step - loss: 1.9764 - accuracy: 0.2998 - val_loss: 1.8288 - val_accuracy: 0.3381
Epoch 2/30
1135/1135 [==============================] - 12s 10ms/step - loss: 1.5860 - accuracy: 0.4683 - val_loss: 1.7827 - val_accuracy: 0.3809
Epoch 3/30
1135/1135 [==============================] - 13s 12ms/step - loss: 1.3177 - accuracy: 0.5716 - val_loss: 1.5866 - val_accuracy: 0.4594
Epoch 4/30
1135/1135 [==============================] - 13s 11ms/step - loss: 1.1049 - accuracy: 0.6531 - val_loss: 1.1299 - val_accuracy: 0.6344
Epoch 5/30
1135/1135 [==============================] - 13s 12ms/step - loss: 0.9339 - accuracy: 0.7138 - val_loss: 0.8935 - val_accuracy: 0.7300
Epoch 6/30
1135/1135 [==============================] - 12s 11ms/step - loss: 0.7883 - accuracy: 0.7619 - val_loss: 0.9025 - val_accuracy: 0.7160
Epoch 7/30
1135/1135 [==============================] - 13s 12ms/step - loss: 0.6781 - accuracy: 0.7968 - val_loss: 0.7641 -

In [13]:
eeg_classifier.save('models/ThoughtViz_obj_eeg.keras')
loss, accuracy = eeg_classifier.evaluate(obj_eeg['x_test'], obj_eeg['y_test'])

179/179 [==============================] - 1s 8ms/step - loss: 1.6777 - accuracy: 0.6725


In [14]:
accuracy

0.672450065612793